# Dispatch Optimisation Problem
1. Define/Create model
    * Abstract or Concrete
2. Populate parameters
3. Decision variables
4. Objective function
5. Constraints
6. Implementation

## 1. Theoretical description

### Formulation of the (economic) Dispatch problem
* Packages used:
    * Pyomo
    * ...

#### 1. Parameters
* PV-specific:
    * *None*
* BESS-specific:
    * Max Storage Capacity
    * Charging Capacity == Discharge Capacity
    * Charge-Efficiency
    * Discharge-Efficiency
    * *Roundtrip Efficiency
    * Mac SoC
    * Min SoC
* Other system-specific parameters:
    * Inverter efficiency
    * PV-BESS system cost
    * Max Grid Volume
* Other parameters that the dispatcher requires:
    * Profit {calculated from the power flows and electricity price at each time-step *t*}
    * State-of-Charge [SoC] {traced/calculated from the SoC[*t-1*] and power flows to/from BESS}

#### 2. Decision Variables
*Decision variables in the dispatch model are not the same as in the global optimisation problem. In the dispatch problem, the decision variables are the different power flows at each time-step *t*
* PV_to_Load
* PV_to_BESS
* PV_curtailment
* PV_to_Grid
* BESS_to_Load
* BESS_to_Grid
* Grid_to_Load
* Grid_to_BESS

#### 3. Objective Function
* MinimiseCost = 
    $$
    min( 
        \sum_{t=o}^{8760} (
            C_{grid}^i[t] * P_{Grid-to-Load}[t]                                                     %cost of electricity from grid
            + C_{PV+BESS}^i[t] * [P_{PV-to-Load}[t] + P_{BESS-to-Load}[t]]                          %cost of electricity from PV-BESS system
            + C_{penalty}^{Curtailment} * P_{Curtailment}[t]                                        %penalty for curtailing PV production
            - C_{grid}^i[t] * [[P_{BESS-to-Grid}[t] + P_{PV-to-Grid}[t] ] - P_{Grid-to-BESS}[t]]    %profit from energy arbitrage
        )
    )
    $$

#### 4. Constraints
* Load fulfilment
    $$
        P_{Load}[t] = P_{PV-to-Load}[t] + P_{BESS-to-Load}[t] + P_{Grid}[t]
    $$
* Solar production
    $$
        P_{PV production}[t] = \frac{P_{PV-to-Load}[t] + P_{PV-to-BESS}[t] + P_{PV-to-Grid}[t]}{\xi_{inverter}} + P_{Curtailment}[t]
    $$
* SoC range
    $$
        SoC^{min} < SoC[t] < SoC^{max} ; \qquad \forall \ \ t
    $$
* SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} ) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * \Delta{t}
        )
    $$
##### 4.1 *Additional* Constraints
* Limiting power flows BESS-to-Grid and PV-to-Grid in energy arbitrage
    $$
        P_{PV-to-Grid}[t] + P_{BESS-to-Grid}[t] <= PowerFlowLimit
    $$
* Limiting the BESS charging to either charge or discharge in the same hour
    $$
        \forall \quad t; \qquad (P_{PV-to-BESS}[t], P_{Grid-to-BESS}[t])*ChargingState\ \ AND\ \ (P_{BESS-to-Load}[t], P_{BESS-to-Grid}[t])*(1-ChargingState)
    $$
    * Subject to:
    * SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} * ChargingState) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * (1-ChargingState) * \Delta{t}
        )
    $$
___
___

The problem will be defined as a worker/tasks set, where the power flows are different "workers" and hours in day are differet "tasks"
The values in each cell has to be the cost, so I will eventually have to make some sort of a generator to populate it quickly

| Flow \ Hour |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 | 10 | ... | 48 | H
|:---------------:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:
| P_PV-to-BESS      | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| P_PV-to-Load      | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| P_BESS-to-Load    | $$p_{f, h}$$ |  $$p_{f, h}$$ |  $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | $$p_{f, h}$$ | ... | - |
| ...               | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | - |
| P_n               | - |  - |  - |  - | - |  - | - | - | - | - | - | - |
**F**

Subject to:
$$
minimize[
    \quad
    \sum_{h=1}^{H}\ (
        \quad
        \sum_{f=1}^{F}\ (\ P_{f, h}\  *\  Cost_{f, h}\ )
        \quad
    )
    \
]
$$

## 2. Implementation

### Implementation notes
* General approach (for now) is to use the *ConcreteModel*
    * If *AbstractModel*, how could the model parameters,variables,etc be populated from the datafile OR the inputs of the model
    * *Why?* --> In the future, the model could be adapted to be compiled and to describe different kinds of and setups of energy systems with different technologies
* Be mindful of how the data is loaded and called in the model to avoid integration errors (like the previous error with numpy)
* Convert the model to kW (previously MW) to avoid issues/error with integer to float conversion
* **OBS!** Since the objective function will be sefined as a double summation function with a general element * factor statement, we need to prepare the cost matrix in a way that reflects energy arbitrage business case. Hence, the "cost" of selling energy to the grid is negative.
* *OBS!*: SoC[*t*] needs a previous value, so we need to artificially override the initial SoC (or implement an SoC for *t* = -1)
* In future implementations/improvements, try converting the time-step to seconds/minutes.
    * *Reason* is to have a more realistic dispatch profile where BESS can charge and discharge in the same hour
* To keep in mind for validation
    * I could also run it with different curtailment penalties and see their effect
    * Underlying difference in optimisation problem when optimising only each time-step vs. optimising a time series window

In [ ]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
## To run the dispatcher later for multiple cases
## (Reference) Cases to run the dispatcher for after valdiation:
power_PV = [0.01, 0.03, 0.05, 0.1, 0.2]
power_BESS = [1, 3, 5, 10, 20]
time_BESS = [0, 1, 2, 3, 4]
optimise_for=   ['Price', 'CO_2_eq']

# Create a new list using list comprehension
referenceCases = []
for i in range(len(power_PV)):
    for j in range(len(power_BESS)):
        for k in range(len(time_BESS)):
            referenceCases.append([power_PV[i], power_BESS[j], power_BESS[j] * time_BESS[k]])

#referenceCases

## Base case for testing/running the dispatcher
## Specifiying system size (as part of the global optimisation problem where the size components are our decision variables)
'''
Since production is modelled for 100MW, is is specified as a scaling factor, where:
0.01    1MW     ==  1 000 kW
0.1     10MW    == 10 000 kW
'''
p_solar=    0.1
## In kW
p_bess=     5000
## In kWh
t_bess=     4
e_bess=     p_bess * t_bess

##### Call in the data, specify the frames of the problem (flows, hours)

In [ ]:
price = pd.read_csv('Data/PriceCurve_SE3_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/pv_sam.csv')
load = pd.read_csv('Data/LoadCurve.csv', sep = ';')

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

#Converting from MW to kW
data['Load']= (data['Load'] * 1000)
#solar data is already in kW, but needs to be multiplied by the scaling factor
data['solar_PV']= (data['solar_PV']) * p_solar
data['Price']= (data['Price'] / 1000)
data['CO_2_eq']= (data['CO_2_eq'] / 1000)

data['Hour']= (data['Hour']).astype('int')

#data.head(48)
#data.head(10)

In [ ]:
data

In [ ]:
#data.describe()

In [ ]:
## Primary data parameters of our scenarios
pv_price= 80                #https://data.nrel.gov/submissions/53 in EUR/kW
bess_price= 200             #https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
# pv_opex= 17                 #EUR/kWh ->reference in excel
# bess_opex= 0.125            #EUR/kWh ->reference in excel
pv_opex= 3
bess_opex= 6

pv_co2= 33                  #kgCO2eq/kW_powerDC ->reference in excel
bess_co2= 100               #kgCO2eq/kWh_capacity ->reference in excel
pv_opex_co2= 0              #kgCO2eq/kW_powerDC ->assumption
bess_opex_co2= 0            #kgCO2eq/kW_powerDC ->assumption
discount_rate= 0.0485       #assumption
lifetime_project= 32        #for the project lifetime
lifetime_bess= 8            #for the BESS lifetime
degradation_rate= 0.025     #assumption (based on reaching 80% SoH in 8 years)

params = {
    'pv_price':         pv_price,
    'bess_price':       bess_price,
    'pv_opex':          pv_opex,
    'bess_opex':        bess_opex,
    'pv_co2':           pv_co2,
    'bess_co2':         bess_co2,
    'pv_opex_co2':      pv_opex_co2,
    'bess_opex_co2':    bess_opex_co2,
    'discount_rate':    discount_rate,
    'lifetime_project': lifetime_project,
    'lifetime_bess':    lifetime_bess,
    'degradation_rate': degradation_rate
}

In [ ]:
## Define the sets/boundaries of the flow/hour table for our values
flows= [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
]

hours= list(range(48))


## Call in the data for the dispatch model
#demand= list(data['Load'][:48])
#pv_production= list(data['solar_PV'][:48])
demand= {}
pv_production= {}
for hour in hours:
    demand[hour]= data['Load'][hour]
    pv_production[hour]= data['solar_PV'][hour]


## Grid availability for each hour is set to 1000 MW, should be more than enough for our model demand and any potential BESS demand for energy arbitrage
grid_production= 1000000


## The intial State-of-Charge for the BESS is 50%
soc_initial = e_bess * 0.5
## Other BESS parameters
efficiency_charge=      0.98
efficiency_discharge=   0.96
efficiency_inverter=    0.97


## Compiling a set for the associated costs of our system flows
costs_keys= []
for flow in flows:
    for hour in hours:
        costs_keys.append((flow, hour))


costs= {}
for i in range(len(costs_keys)):
    if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
        costs[costs_keys[i]]= data['Price'][costs_keys[i][1]]
    elif costs_keys[i][0] == 'P_PV_to_Grid':
        costs[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]]) - params['pv_opex']/1000)
    elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
        costs[costs_keys[i]]= params['pv_opex']/1000
    elif costs_keys[i][0] =='P_BESS_to_Grid':
        costs[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]]) - params['bess_opex']/1000)
    elif costs_keys[i][0] == 'P_BESS_to_Load':
        costs[costs_keys[i]]= params['bess_opex']/1000
    elif costs_keys[i][0] == 'P_PV_curtailment':
        costs[costs_keys[i]]= 1000/1000
    else:
        continue
#costs

In [ ]:
#hours[2:9]

##### Define model

In [ ]:
## Initialise the model
model= pe.ConcreteModel()

##### Define Sets

In [ ]:
## Defining the sets for our variables
model.flows= pe.Set(initialize= flows, ordered= True)
model.hours= pe.Set(initialize= hours, ordered= True)

In [ ]:
#model.display()

##### Define Parameters

In [ ]:
## Defining parameters
model.grid_production=      pe.Param(initialize= grid_production)
model.demand=               pe.Param(model.hours, initialize= demand)
model.pv_production=        pe.Param(model.hours, initialize= pv_production)

model.Efficiency_charge=    pe.Param(model.hours, initialize= efficiency_charge)
model.Efficiency_inverter=  pe.Param(model.hours, initialize= efficiency_discharge)
model.Efficiency_discharge= pe.Param(model.hours, initialize= efficiency_inverter)

#model.SoCmin=               pe.Param(model.hours, initialize= (e_bess*0.1))
#model.SoCmax=               pe.Param(model.hours, initialize= (e_bess*0.9))
#model.SoC=                  pe.Param(model.hours, initialize= (e_bess*0.5))

model.SoCmin=               pe.Param(initialize= (e_bess*0.1))
model.SoCmax=               pe.Param(initialize= (e_bess*0.9))
model.SoCinitial=           pe.Param(initialize= soc_initial)
model.arbitrageLimit=       pe.Param(model.hours, initialize= (p_solar * 1000000))
model.costs=                pe.Param(model.flows, model.hours, initialize=costs, default= 1)

In [ ]:
# model.display()

In [ ]:
#model.grid_production.value

In [ ]:
#model.costs['P_PV_to_Load', 24]

In [ ]:
# model.SoC[0]

##### Variables

In [ ]:
#model.hours.data()

In [ ]:
## Defining Variables
model.p=    pe.Var(model.flows, model.hours, domain= pe.NonNegativeReals)
model.SoC=  pe.Var(model.hours, domain= pe.Reals, bounds= (model.SoCmin, model.SoCmax))

In [ ]:
#model.p.display()

In [ ]:
#model.SoC.display()

##### Objective

In [ ]:
## Objective function
minCosts = sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours)
model.objective= pe.Objective(sense= pe.minimize, expr= minCosts)


# minCosts= (
#     model.p['P_PV_to_Load', t] * model.costs['P_PV_to_Load', t]  \
#     + model.p['P_BESS_to_Load', t] * model.costs['P_BESS_to_Load', t]    \
#     + model.p['P_Grid_to_Load', t] * model.costs['P_Grid_to_Load', t]    \
#     + model.p['P_PV_curtailment', t] * model.costs['P_PV_curtailment', t]\
#     + model.p['P_Grid_to_BESS', t] * model.costs['P_Grid_to_BESS', t]    \
#     - model.p['P_PV_to_Grid', t] * model.costs['P_PV_to_Grid', t]        \
#     - model.p['P_BESS_to_Grid', t] * model.costs['P_BESS_to_Grid', t]    \
#     for t in model.hours
# )

# model.objective=    pe.Objective(
#     sense=  pe.minimize,
#     expr=   minCosts
# )

In [ ]:
#model.objective.display()

In [ ]:
#model.pprint()

##### Constraints & Relations

In [ ]:
## Constraints
loadFullfilment=    {t: model.p['P_PV_to_Load', t] + model.p['P_BESS_to_Load', t] + model.p['P_Grid_to_Load', t] == model.demand[t] for t in model.hours}
pvProduction=       {t: model.p['P_PV_to_Load', t] + model.p['P_PV_to_BESS', t] + model.p['P_PV_to_Grid', t] + model.p['P_PV_curtailment', t] == model.pv_production[t] for t in model.hours}
arbitrageFlow=      {t: (model.p['P_BESS_to_Grid',t] + model.p['P_PV_to_Grid',t]) <= model.arbitrageLimit[t] for t in model.hours}
gridFlow=           {t: model.p['P_Grid_to_Load', t] + model.p['P_Grid_to_BESS', t] <= model.grid_production for t in model.hours}

In [ ]:
## Old/Redundant

# socTracker=         {
#     t: model.SoC[t] == model.SoC[t-1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (model.Efficiency_charge * model.Efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (model.Efficiency_discharge * model.Efficiency_inverter)) for t in model.hours
# }
# Additional constrains, first run without
#chargeState=
#arbitrageLimit=

#socRangeMin=        e_bess * 0.1 <= (model.SoC[t] for t in hours)
#socRangeMin=        {t: model.SoC[t] >= model.SoCmin[t] for t in model.hours}
#socRangeMax=        {t: model.SoC[t] <= model.SoCmax[t] for t in model.hours}

In [ ]:
def storage_state(model, t):
            if t == model.hours.first():
                return model.SoC[t] == model.SoCinitial + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))
            else:
                return model.SoC[t] == model.SoC[t-1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))
model.charge_state = pe.Constraint(model.hours, rule = storage_state)

In [ ]:
model.demandRule=       pe.Constraint(model.hours, expr= loadFullfilment)
model.pvProductionRule= pe.Constraint(model.hours, expr= pvProduction)
model.arbitrageRule=    pe.Constraint(model.hours, expr= arbitrageFlow)
model.gridRule=         pe.Constraint(model.hours, expr= gridFlow)
#model.socMinRule=       pe.Constraint(model.hours, expr= socRangeMin)
#model.socMaxRule=       pe.Constraint(model.hours, expr= socRangeMax)
#model.socTracking=      pe.Constraint(model.hours, expr= socTracker)


#model.pprint()

In [ ]:
solver= po.SolverFactory('glpk')
results= solver.solve(model, tee= False)
#results= solver.solve(model, tee= True)

In [ ]:
#results

In [ ]:
print(pe.value(model.objective))

In [ ]:
# for var in model.component_data_objects(pe.Var, active=True):
#     if 'SoC' in var.name:
#         print(f"{var.name}: {pe.value(var)}")
#         #print('------------------------------------------------------------')
#     else:
#         pass

# print(' '*150)
# print('#'*150)
# print('#'*150)
# print('#'*150)
# print(' '*150)
# for var in model.component_data_objects(pe.Var, active=True):
#     if 'SoC' in var.name:
#         pass
#     else:
#         #print(f"{var.name[-3:-1]}: {pe.value(var)}")
#         print(f"{var.name}: {pe.value(var)}")
#         #print('------------------------------------------------------------')

In [ ]:
#model.p['P_PV_to_Load', 1].value

## Exporting results

In [ ]:
flows.append('SoC')
#flows

In [ ]:
df_test= pd.DataFrame(index= hours, columns= flows)
#df_test

In [ ]:
for flow in flows:
    for hour in hours:
        if flow != 'SoC':
            df_test[flow][hour]= model.p[flow, hour].value
        else:
            df_test['SoC'][hour]= model.SoC[hour].value
df_test= df_test.reset_index()
df_test= df_test.rename(columns={'index': 'Hour'})
#df_test

In [ ]:
#df_test.info()

In [ ]:
# Convert columns to float
for flow in flows:
    df_test[flow] = df_test[flow].astype(float)

df_test['sum_power_flows'] = df_test.P_PV_to_Load + df_test.P_BESS_to_Load + df_test.P_Grid_to_Load
df_test['sum_power_flows'] = df_test['sum_power_flows'].astype(float)
df_test['SoC']= (df_test['SoC']/e_bess) *100
# Now, check the data types
#print(df_test.dtypes)


In [ ]:
df_test.plot(
    x= 'Hour',
    y= [
        'P_PV_to_Load',
        'P_Grid_to_Load',
        'P_BESS_to_Load',
        #'P_Grid_to_BESS',
        #'P_BESS_to_Grid',
        'sum_power_flows'
        ]
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(
    df_test.Hour,
    df_test.sum_power_flows,
    linestyle= 'dashdot',
    linewidth= 2
)

In [ ]:
plt.stackplot(
    df_test.Hour,
    df_test.P_PV_to_Load,
    df_test.P_Grid_to_Load,
    df_test.P_BESS_to_Load,
    labels=[
        'P_PV_to_Load',
        'P_Grid_to_Load',
        'P_BESS_to_Load',
    ]
)
plt.plot(df_test.Hour, df_test.sum_power_flows, linestyle= 'dashdot', linewidth= 2)
plt.legend(loc='upper left')


In [ ]:
# Assuming 'Hour', 'P_PV_to_BESS', 'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_BESS', 'SoC' are columns in df_test
fig, ax1 = plt.subplots()

# Primary y-axis
ax1.plot(df_test['Hour'], df_test[['P_PV_to_BESS', 'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_BESS']])
ax1.set_xlabel('Hour')
ax1.set_ylabel('Flows [kW]')

# Create a twin Axes sharing the xaxis
ax2 = ax1.twinx()
ax2.plot(df_test['Hour'], df_test['SoC'], 'b', linestyle= 'dashdot')
ax2.set_ylabel('SoC [%]', color='b')

plt.show()


In [ ]:
# Assuming 'Hour', 'P_PV_to_BESS', 'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_BESS', 'SoC' are columns in df_test
fig, ax1 = plt.subplots()

# Primary y-axis
ax1.plot(df_test['Hour'], df_test['P_PV_to_BESS'], label='P_PV_to_BESS')
ax1.plot(df_test['Hour'], df_test['P_BESS_to_Load'], label='P_BESS_to_Load')
ax1.plot(df_test['Hour'], df_test['P_BESS_to_Grid'], label='P_BESS_to_Grid')
ax1.plot(df_test['Hour'], df_test['P_Grid_to_BESS'], label='P_Grid_to_BESS')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Flows in kW')

# Create a twin Axes sharing the xaxis
ax2 = ax1.twinx()
ax2.plot(df_test['Hour'], df_test['SoC'], 'b', label='SoC', linestyle= 'dashdot')
ax2.set_ylabel('SoC', color='b')

# Combine legends for both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()


In [ ]:
profile= pd.read_csv(
    'Results/Batch4/CaseB/dispatch_10_5000_4_price_old.csv.csv',
    sep= ',',
    index_col=False,
)
profile['SoC']= (profile['SoC']/20000)*100
profile

In [ ]:
week=   25

In [ ]:
fig= plt.figure(
    figsize= (25,8),
)
title= plt.title('P_PV: 10 MW; P_BESS: 5 MW; T_BESS: 4 hr')

plt.stackplot(
    profile[:168].Hour,
    profile[:168].P_PV_to_Load,
    profile[:168].P_Grid_to_Load,
    profile[:168].P_BESS_to_Load,
    labels=[
        'P_PV_to_Load',
        'P_Grid_to_Load',
        'P_BESS_to_Load',
    ],
)
plt.plot(profile[:168].Hour, profile[:168].sum_power_flows, linestyle= 'dashdot', linewidth= 2)
plt.legend(loc='upper left')


In [ ]:
fig= plt.figure(
    figsize= (25,8)
)
title= plt.title('P_PV: 10 MW; P_BESS: 5 MW; T_BESS: 4 hr')

plt.stackplot(
    profile[(week * 168):(week + 1) * 168].Hour,
    profile[(week * 168):(week + 1) * 168].P_PV_to_Load,
    profile[(week * 168):(week + 1) * 168].P_Grid_to_Load,
    profile[(week * 168):(week + 1) * 168].P_BESS_to_Load,
    labels=[
        'P_PV_to_Load',
        'P_Grid_to_Load',
        'P_BESS_to_Load',
    ],
)
plt.plot(profile[(week * 168):(week + 1) * 168].Hour, profile[(week * 168):(week + 1) * 168].sum_power_flows, linestyle= 'dashdot', linewidth= 2)
plt.legend(loc='upper left')


In [ ]:
fig= plt.figure(
    figsize= (25,8)
)
title= plt.title('P_PV: 10 MW; P_BESS: 5 MW; T_BESS: 4 hr')

plt.stackplot(
    profile[(week * 168):(week + 1) * 168].Hour,
    profile[(week * 168):(week + 1) * 168].P_PV_to_Load,
    profile[(week * 168):(week + 1) * 168].P_PV_to_Grid,
    profile[(week * 168):(week + 1) * 168].P_PV_to_BESS,
    labels=[
        'P_PV_to_Load',
        'P_PV_to_Grid',
        'P_PV_to_BESS',
    ],
)
plt.plot(profile[(week * 168):(week + 1) * 168].Hour, profile[(week * 168):(week + 1) * 168].sum_power_flows, linestyle= 'dashdot', linewidth= 2)
plt.legend(loc='upper left')


In [ ]:
# Assuming 'Hour', 'P_PV_to_BESS', 'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_BESS', 'SoC' are columns in profile
fig, ax1 = plt.subplots(figsize=(25,8))
title= plt.title('P_PV: 10 MW; P_BESS: 5 MW; T_BESS: 4 hr')

# Primary y-axis
ax1.plot(profile[:168]['Hour'], profile[:168]['P_PV_to_BESS'], label='P_PV_to_BESS')
ax1.plot(profile[:168]['Hour'], profile[:168]['P_BESS_to_Load'], label='P_BESS_to_Load')
ax1.plot(profile[:168]['Hour'], profile[:168]['P_BESS_to_Grid'], label='P_BESS_to_Grid')
ax1.plot(profile[:168]['Hour'], profile[:168]['P_Grid_to_BESS'], label='P_Grid_to_BESS')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Flows in kW')

# Create a twin Axes sharing the xaxis
ax2 = ax1.twinx()
ax2.plot(profile[:168]['Hour'], profile[:168]['SoC'], 'b', label='SoC', linestyle= 'dashdot')
ax3 = ax1.twinx()
ax3.plot(data[:168]['Hour'], data[:168]['Price'], 'red', label='Price', linestyle= ':')

ax2.set_ylabel('SoC', color='b')
ax2.tick_params(colors= 'b')
ax3.set_ylabel('Price', color= 'r')
ax3.tick_params(colors= 'r')

# Combine legends for both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()


In [ ]:
# Assuming 'Hour', 'P_PV_to_BESS', 'P_BESS_to_Load', 'P_BESS_to_Grid', 'P_Grid_to_BESS', 'SoC' are columns in profile
fig, ax1 = plt.subplots(figsize=(25,8))
title= plt.title('P_PV: 10 MW; P_BESS: 5 MW; T_BESS: 4 hr')

# Primary y-axis
ax1.plot(profile[(week * 168):(week + 1) * 168]['Hour'], profile[(week * 168):(week + 1) * 168]['P_PV_to_BESS'], label='P_PV_to_BESS')
ax1.plot(profile[(week * 168):(week + 1) * 168]['Hour'], profile[(week * 168):(week + 1) * 168]['P_BESS_to_Load'], label='P_BESS_to_Load')
ax1.plot(profile[(week * 168):(week + 1) * 168]['Hour'], profile[(week * 168):(week + 1) * 168]['P_BESS_to_Grid'], label='P_BESS_to_Grid')
ax1.plot(profile[(week * 168):(week + 1) * 168]['Hour'], profile[(week * 168):(week + 1) * 168]['P_Grid_to_BESS'], label='P_Grid_to_BESS')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Flows in kW')

# Create a twin Axes sharing the xaxis
ax2 = ax1.twinx()
ax2.plot(profile[(week * 168):(week + 1) * 168]['Hour'], profile[(week * 168):(week + 1) * 168]['SoC'], 'b', label='SoC', linestyle= 'dashdot')
ax3 = ax1.twinx()
ax3.plot(data[(week * 168):(week + 1) * 168]['Hour'], data[(week * 168):(week + 1) * 168]['Price'], 'red', label='Price', linestyle= ':')
ax2.set_ylabel('SoC', color='b')
ax2.tick_params(colors= 'b')
ax3.set_ylabel('Price', color= 'r')
ax3.tick_params(colors= 'r')

# Combine legends for both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()
